In [1]:
from activeLearning_with_Fastfit.human_annotator_with_hint import MySQLDataHandler, Annotation, ModelTraining, ActiveLearning

In [2]:
from datasets import Dataset, Features, Value,load_dataset,DatasetDict


In [3]:
import pandas as pd

In [4]:
uname = 'zijianan'

In [5]:
ori_table_name = 'blm_tweets'

In [6]:
human_annotated_table_name = 'blm_annotated_tweets'

In [7]:
db_handler = MySQLDataHandler('localhost', 'zijianan', '12Q3qeqs,', '5_category')

In [8]:
save_path = '/data/blm_model/'+uname+'_'

In [9]:
labels = ['pos','neg','unknown']

### random sample

In [10]:
query = f'''
SELECT * FROM {ori_table_name} ORDER BY RAND() LIMIT 50;
'''
random_blm_data = db_handler.fetch_data(query=query)

In [12]:
random_blm_data

id                rt_id               rt_uid  \
0   2331411  1349188813525053440             24733117   
1   1565474  1131772627167662087             77646168   
2   2682674  1532197311262208000           3257608936   
3   1928334  1387712611203133440             21950364   
4   2566734   872567663414747136             90480218   
5    226289  1298460036587335680             16245840   
6   2598810  1305929584643387395             30699754   
7   3775959   504840215924846592            279390084   
8   3082334  1405918348416847883           2482515498   
9   1405982   570005217485426688             16967845   
10  3841924   415254507640348672            427799256   
11  1742560   519965171465203712             19542622   
12  3566560  1622716262223675408              6782762   
13  1012054   929404074075631616             14894518   
14  1476442    69992490010279936              7782542   
15  2342842  1523289879207366657              1917731   
16  2434042   917063260549124097           1687584320   
17   253237   534866192922853377             16331010   
18  2208757  1564664547440484354            232901331   
19  3759157  1391876635998277632            809115114   
20  2446087  1110901274415906819             84734907   
21  2849495  1049426150693457920   921107749471219712   
22  1563455  1088515741282754560             10252962   
23  3370365   541495504186507264             14222536   
24  2463297  1305224757202292737            329221989   
25  2756097   193021197888389120             16348549   
26  3648897   816857467615383553            138190051   
27  4124097   828693960013049856           4327242015   
28  1243988  1305848846933778433            472987747   
29   979937  1384298204783542273            907139600   
30  3524100  1002014859213656065   711026018048286722   
31   772519  1331007116971130881             14177942   
32  3627379  1459233134109765633            299896657   
33  3167727   584033486958370816            315833613   
34  1579199  1286729620893700098             15369276   
35  3494399  1498855657675628551           1392282998   
36  3994898   947101098975940608             28785486   
37  2760884   547569216157913088             15864446   
38  3743906  1269758561720156160             50055701   
39  4054878   784200319622258688             20058011   
40  4328478    84467466264260608             22056192   
41  3169196   636435604822077440             18014620   
42  3989036   839893470470889473             17023380   
43  2213224   223617994742448130            465763204   
44  1995285  1382082435278647301   984500402350641152   
45  3576432   654802199021654016             14717197   
46   926068  1678367877899091968  1001663707045421058   
47  1456033  1405929145477992450             50735431   
48  3999344  1011737013958029313            226194552   
49  2353527   542437661433012225            123478177   

                                        modeling_text  
0   manhunt intensifies as authorities warn some r...  
1   🔱#topmeme award goes to @irrepressably for thi...  
2   california released a 600-page reparations rep...  
3   how do you describe today’s republican party?\...  
4   i'll be joining my friend @seanhannity tonight...  
5   melania is often described as useless but plea...  
6   details on the settlement that the city of lou...  
7   judicial system's blessing of police use of ex...  
8   this #juneteenth we must go beyond recognizing...  
9   over 1 in 7 women, nearly 18 million, lived in...  
10  research says: encouraging girls is key in clo...  
11  this is why people hate politics. rt @wltx: (a...  
12  in 100 years, if the woke have their way, peop...  
13  why we need a broad-based movement, like the w...  
14  wash post editorial bd calls for moratorium on...  
15  the possibility that the supreme court will ov...  
16  if a girl tells you that you look like troye s...  
17  floyd mayweather, twerk-aholic: 10 girls, 1 fl...  
18  pa senate candidate outs himself 

In [15]:

annotations = Annotation.run_annotation(random_blm_data,labels,'modeling_text')

HTML(value='0 of 50 Examples annotated, Current Position: 0 ')

Output()

In [17]:
random_blm_data['label'] = annotations['label']

In [10]:
random_blm_data = pd.read_csv('blm_test.csv')

In [12]:
validate_data = pd.read_csv('/data/zijian/negative_tweet_examples.csv')

In [15]:
rt_ids = validate_data['rt_id'].tolist()
rt_ids_str = ', '.join([str(id) for id in rt_ids])

In [16]:
query = f"SELECT * FROM {ori_table_name} WHERE rt_id IN ({rt_ids_str}) ORDER BY RAND() LIMIT 200;"
neg_vali = db_handler.fetch_data(query=query)

In [17]:
query = f"SELECT * FROM {ori_table_name} WHERE rt_id NOT IN ({rt_ids_str}) ORDER BY RAND() LIMIT 200;"
pos_vali = db_handler.fetch_data(query=query)

In [18]:
neg_vali['label'] = 'neg'
pos_vali['label'] = 'pos'

In [19]:
random_blm_data_vali = pd.concat([neg_vali,pos_vali]).reset_index(drop=True)

In [16]:
# vali_per = 0.4
# random_blm_data_vali = random_blm_data.groupby('label').apply(lambda x: x.sample(frac=vali_per, random_state=42)).reset_index(drop=True)


In [20]:
dataset = Dataset.from_pandas(random_blm_data[random_blm_data['label'].isin(['neg','pos'])][['modeling_text','label']])

In [21]:
dataset = DatasetDict({
    'train': dataset  # Set the 'train' split
})

In [22]:
dataset['validation'] = Dataset.from_pandas(random_blm_data_vali[['modeling_text','label']])

In [23]:
dataset["test"] = dataset["validation"]

In [24]:
save_path

'/data/blm_model/zijianan_'

In [25]:
save_path = save_path + "blm_random_0"

In [26]:

training_instance = ModelTraining(dataset=dataset, \
                                  model_name_or_path='launch/POLITICS', \
                                  label_column_name='label', \
                                  text_column_name='modeling_text',\
                                  save_path=save_path, \
                                  num_train_epochs=10)
model_0 = training_instance.train_model()
model_0.to('cpu')
import torch
torch.cuda.empty_cache()

05/08/2024 10:50:30 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True


[WARNING|modeling_utils.py:4172] 2024-05-08 10:50:31,529 >> Some weights of RobertaModel were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/29 [00:00<?, …

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/400 [00:00<?,…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/400 [00:00<?,…

...

05/08/2024 10:50:41 - ERROR - wandb.jupyter - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: zijianan. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[WARNING|modeling_utils.py:1188] 2024-05-08 10:50:53,246 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =     3.9178
  train_runtime            = 0:00:14.57
  train_samples            =         29
  train_samples_per_second =     19.904
  train_steps_per_second   =      1.373


***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.5325
  eval_loss               =     5.9804
  eval_runtime            = 0:00:00.82
  eval_samples            =        400
  eval_samples_per_second =    486.078
  eval_steps_per_second   =     15.798
{'eval_loss': 5.980385780334473, 'eval_accuracy': 0.5325, 'eval_runtime': 0.8229, 'eval_samples_per_second': 486.078, 'eval_steps_per_second': 15.798, 'epoch': 10.0, 'eval_samples': 400}
Accuracy: 53.2%


### active learning

In [24]:

# query = f'''
# SELECT * FROM {ori_table_name} ORDER BY RAND() LIMIT 1000;
# '''
# inference_data = db_handler.fetch_data(query=query)

In [27]:
inference_data = random_blm_data_vali.drop('label',axis=1)

In [28]:
save_path = '/data/blm_model/zijianan_blm_random_'

In [32]:
active_learning_instance = ActiveLearning(model_name_or_path=save_path, \
                                  label_column_name='label', \
                                  text_column_name='modeling_text',\
                                  theta=10,
                                  i=3,
                                  vali_data=dataset["validation"],
                                  inference_datasets=inference_data,
                                  tokenizer_name='roberta-large',
                                  uname=uname,
                                  labels=['neg','pos'])

[ERROR|base.py:1089] 2024-05-08 11:15:55,104 >> The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSe

In [31]:
t1 = active_learning_instance.perform_epochs()

Epoch 1/3


[ERROR|base.py:1089] 2024-05-08 10:51:32,917 >> The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSe

HTML(value='0 of 10 Examples annotated, Current Position: 0 ')

Output()

05/08/2024 10:51:52 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True


...

Running tokenizer on dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

05/08/2024 10:53:00 - WARNING - accelerate.utils.other - Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/zijianan/miniconda3/envs/fu/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[WARNING|modeling_utils.py:1188] 2024-05-08 10:53:00,728 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =     3.4587
  train_runtime            = 0:00:01.39
  train_samples            =         10
  train_samples_per_second =     71.442
  train_steps_per_second   =      7.144


***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.5275
  eval_loss               =     6.5381
  eval_runtime            = 0:00:00.80
  eval_samples            =        400
  eval_samples_per_second =    496.641
  eval_steps_per_second   =     16.141
{'eval_loss': 6.5380859375, 'eval_accuracy': 0.5275, 'eval_runtime': 0.8054, 'eval_samples_per_second': 496.641, 'eval_steps_per_second': 16.141, 'epoch': 10.0, 'eval_samples': 400}
Accuracy: 52.8%
